<a href="https://colab.research.google.com/github/ksherdy/udacity-cs344-colab/blob/master/notebook/MyFirstCuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-390' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-410' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-415' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-418' for glob 'nvidia*'
Note, selecting 'nvidi

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-10.0


--2020-06-28 21:39:11--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.57.194
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.57.194|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?je2p2oaENEckuIt2wKX0FMrQAb21bhjO8W35MHJL26X9LVwgVgrvv_TQTFiYaEqOd_HMZVQ9BHV4JsOz_UN38YPJ2kKLAKfzgor8wRbGM4kREnEt865ZqewDyFE9Dv5TDnDwlwfnyNF8yvOedGcw0jSn8bJ0d2Ehr2dtLA-r2R7yPR7rY7X5YFNgBN-u3ap4OEOE05RRE-zmANevmUM [following]
--2020-06-28 21:39:12--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?je2p2oaENEckuIt2wKX0FMrQAb21bhjO8W35MHJL26X9LVwgVgrvv_TQTFiYaEqOd_HMZVQ9BHV4JsOz_UN38YPJ2kKLAKfzgor8wRbGM4kREnEt865ZqewD

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-oo2f64bi
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-oo2f64bi
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=4df3191e9f87dfc3c2f430ae9697649a562a855aeaa5df12275441af463a4bb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-7heoul0o/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>

__global__ void square(float * d_in, float * d_out) {
    int i = threadIdx.x;
    float f = d_in[i];
    d_out[i] = f*f;
}

int main() {

    int ARRAY_SIZE = 64;
    int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);

    float h_in[ARRAY_SIZE];

    for(int i=0;i<ARRAY_SIZE;i++) {
        h_in[i] = i+1;
    }

    float * d_in;
    float * d_out;

    cudaMalloc((void **) &d_in, ARRAY_BYTES);
    cudaMalloc((void **) &d_out, ARRAY_BYTES);

    cudaMemcpy(d_in, h_in, ARRAY_BYTES, cudaMemcpyHostToDevice);

    square<<<1,ARRAY_SIZE>>>(d_in, d_out);

    cudaMemcpy(h_in, d_out, ARRAY_BYTES, cudaMemcpyDeviceToHost);

    for(int i=0;i<ARRAY_SIZE;i++) {
        printf("%f", h_in[i]);
        printf(((i % 4) != 3) ? "\t" : "\n");
    }

    cudaFree(d_in);
    cudaFree(d_out);

    return 0;
}


1.000000	4.000000	9.000000	16.000000
25.000000	36.000000	49.000000	64.000000
81.000000	100.000000	121.000000	144.000000
169.000000	196.000000	225.000000	256.000000
289.000000	324.000000	361.000000	400.000000
441.000000	484.000000	529.000000	576.000000
625.000000	676.000000	729.000000	784.000000
841.000000	900.000000	961.000000	1024.000000
1089.000000	1156.000000	1225.000000	1296.000000
1369.000000	1444.000000	1521.000000	1600.000000
1681.000000	1764.000000	1849.000000	1936.000000
2025.000000	2116.000000	2209.000000	2304.000000
2401.000000	2500.000000	2601.000000	2704.000000
2809.000000	2916.000000	3025.000000	3136.000000
3249.000000	3364.000000	3481.000000	3600.000000
3721.000000	3844.000000	3969.000000	4096.000000

